In [1]:
import pandas as pd

In [78]:
df = pd.read_csv("Biosample_all.tsv", sep="\t").fillna("")

/Users/cjm/repos/mondo-analysis/venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (13,14,16,18,19,23,26,27,28,29,30,34,35,36,37,39,41,42,43,44,45,52,57,62,70,72,73,76,77,78,79,86,98,99,100,103,104,105,106,111,119,124,133,134,135,142,144,146,149,165,191,208,229,233,252,253,254) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [79]:
df[0:5]

,BIOSAMPLE_ID,BIOSAMPLE_NAME,ADD_DATE,MOD_DATE,MOD_BY,DESCRIPTION,ECOSYSTEM,ECOSYSTEM_CATEGORY,ECOSYSTEM_TYPE,ECOSYSTEM_SUBTYPE,...,DEPTH2,ALTITUDE,ALTITUDE2,SOLUBLE_IRON_MICROMOL,BICARBONATE_MILLIMOL,H2S_MILLIMOL,H2S_PRESENT,IRRADIANCE,OXYGEN_PRESENCE,METHANE_CONC_MILLIMOL
0,173480,Wine grape associated microbial communities fr...,09-DEC-17 11.15.52.119000000 PM,05-JAN-18 12.58.38.000000000 PM,102151,Wine grape associated microbial communities fr...,Host-associated,Plants,Phyllosphere,Caulosphere,...,,,,,,,,,,
1,173774,Fermented milk associated microbial communitie...,09-DEC-17 11.36.30.369000000 PM,05-JAN-18 12.54.38.000000000 PM,102151,Fermented milk (koumiss) associated microbial ...,Engineered,Food production,Dairy products,,...,,,,,,,,,,
2,173775,Fermented milk associated microbial communitie...,09-DEC-17 11.36.34.360000000 PM,05-JAN-18 12.54.43.000000000 PM,102151,Fermented milk (koumiss) associated microbial ...,Engineered,Food production,Dairy products,,...,,,,,,,,,,
3,173776,Fermented milk associated microbial communitie...,09-DEC-17 11.36.38.315000000 PM,05-JAN-18 12.54.43.000000000 PM,102151,Fermented milk (koumiss) associated microbial ...,Engineered,Food production,Dairy products,,...,,,,,,,,,,
4,173784,Fermented milk associated microbial communitie...,09-DEC-17 11.37.10.939000000 PM,05-JAN-18 12.54.43.000000000 PM,102151,Fermented milk (koumiss) associated microbial ...,Engineered,Food production,Dairy products,,...,,,,,,,,,,


In [80]:
elevels = ['ECOSYSTEM', 'ECOSYSTEM_CATEGORY', 'ECOSYSTEM_TYPE', 'ECOSYSTEM_SUBTYPE', 'SPECIFIC_ECOSYSTEM']
df[elevels][0:10]

,ECOSYSTEM,ECOSYSTEM_CATEGORY,ECOSYSTEM_TYPE,ECOSYSTEM_SUBTYPE,SPECIFIC_ECOSYSTEM
0,Host-associated,Plants,Phyllosphere,Caulosphere,
1,Engineered,Food production,Dairy products,,
2,Engineered,Food production,Dairy products,,
3,Engineered,Food production,Dairy products,,
4,Engineered,Food production,Dairy products,,
5,Engineered,Food production,Dairy products,,
6,Engineered,Food production,Dairy products,,
7,Environmental,Aquatic,Freshwater,River,
8,Environmental,Aquatic,Freshwater,River,
9,Environmental,Aquatic,Freshwater,Lake,Sediment


In [81]:
## Add paths

def mkpath(row):
    s = False
    toks = []
    for l in elevels[::-1]:
        v = row[l]
        if (v == "" or v == "Unclassified") and not s:
            continue
        else:
            toks.append(v)
    toks.reverse()
    #print(str(toks))
    return ":".join(toks)



In [82]:
df['path'] = df.apply(lambda r : mkpath(r), axis=1)

In [83]:
df['path'][0:10]

0    Host-associated:Plants:Phyllosphere:Caulosphere
1          Engineered:Food production:Dairy products
2          Engineered:Food production:Dairy products
3          Engineered:Food production:Dairy products
4          Engineered:Food production:Dairy products
5          Engineered:Food production:Dairy products
6          Engineered:Food production:Dairy products
7             Environmental:Aquatic:Freshwater:River
8             Environmental:Aquatic:Freshwater:River
9     Environmental:Aquatic:Freshwater:Lake:Sediment
Name: path, dtype: object

In [84]:
## num distinct paths
len(set(df['path']))

461

## Find primitives that exist at multiple levels


In [85]:
for c in elevels:
    sc = set(df[c])
    sc.discard("")
    sc.discard("Unclassified")
    for d in elevels:
        if c <= d:
            continue
        sd = set(df[d])
        ixn = sc.intersection(sd)
        if len(ixn) > 0:
            print(' * {} /\ {} = {}'.format(c,d,ixn))
        

 * ECOSYSTEM_TYPE /\ ECOSYSTEM_CATEGORY = {'Cnidaria', 'Echinodermata', 'Wastewater', 'Fungi'}
 * ECOSYSTEM_TYPE /\ ECOSYSTEM_SUBTYPE = {'Sediment', 'Wood', 'Wastewater', 'Soil', 'Nodule', 'Volcanic'}
 * ECOSYSTEM_SUBTYPE /\ ECOSYSTEM_CATEGORY = {'Wastewater', 'Endosymbionts'}
 * SPECIFIC_ECOSYSTEM /\ ECOSYSTEM_CATEGORY = {'Bioreactor'}
 * SPECIFIC_ECOSYSTEM /\ ECOSYSTEM_TYPE = {'Sediment', 'Tissue'}
 * SPECIFIC_ECOSYSTEM /\ ECOSYSTEM_SUBTYPE = {'Microbialites', 'Sediment', 'Fecal', 'Alkaline', 'Mine'}


## Translate TSV into a graph

 * Each node is a primitive
 * parents = primitives in the level above
 * children = primitives in the level below

In [86]:
import networkx as nx

In [101]:
## Turn dataframe into a graph
node2paths = {}
def df2g(df):
    G = nx.DiGraph()
    for ix, row in df.iterrows():
        path = ""
        for i in range(1,len(elevels)):
            p = row[elevels[i-1]]
            c = row[elevels[i]]
            path += ">" + p
            if c != "" and c != "Unclassified":
                G.add_edge(p,c)
                if c not in node2paths:
                    node2paths[c] = set()
                node2paths[c].add(path)
    return G

G = df2g(df)          

In [88]:
G.predecessors('Solid waste')

['Engineered']

In [89]:
G.successors('Solid waste')

['Solid animal waste', 'Landfill', 'Composting', 'Swine wastewater']

In [90]:
## Total primitives
len(G.nodes())

369

## Primitives that exist in more than one place

In [102]:
for n in G.nodes_iter():
    ps = [p for p in G.predecessors(n) if p != "Unclassified"]
    if len(ps) > 1:
        print(" * {} parents: {}".format(n, ps))
        for path in node2paths[n]:
            print("     * PATH: {}".format(path))

 * Sediment parents: ['Lake', 'Freshwater', 'Hydrothermal vents', 'Aquatic', 'Coastal', 'Oceanic', 'Sediment', 'Hot (42-90C)', 'Thermal springs', 'Wetlands', 'Pond', 'Intertidal zone', 'Geologic', 'Hypersaline', 'Lentic', 'Neritic zone', 'Saline', 'Marine', 'Lotic', 'Alkaline', 'Cold seeps', 'Subtidal zone', 'Floodplain']
     * PATH: >Environmental>Aquatic>Thermal springs>Hot (42-90C)
     * PATH: >Environmental>Aquatic>Marine>Wetlands
     * PATH: >Environmental>Aquatic>Freshwater>Lotic
     * PATH: >Environmental>Terrestrial>Geologic
     * PATH: >Environmental>Aquatic>Marine>Neritic zone
     * PATH: >Environmental>Aquatic>Freshwater
     * PATH: >Environmental>Aquatic>Non-marine Saline and Alkaline>Saline
     * PATH: >Environmental>Aquatic>Marine>Hydrothermal vents
     * PATH: >Environmental>Aquatic>Marine>Coastal
     * PATH: >Environmental>Aquatic
     * PATH: >Environmental>Aquatic>Thermal springs
     * PATH: >Environmental>Terrestrial>Floodplain
     * PATH: >Environmental>

In [43]:
import networkx as nx

for i in range(0,elevels):
    li = elevels[i]
    for ci in set(df[ci]):
        
    for j in range(i,elevels):
        lj = elevels[j]
        

IndentationError: expected an indented block (<ipython-input-43-cd55e4ad5d63>, line 7)

## Depth

In [108]:
depth_df = df[df.DEPTH != ""]
depth_df

,BIOSAMPLE_ID,BIOSAMPLE_NAME,ADD_DATE,MOD_DATE,MOD_BY,DESCRIPTION,ECOSYSTEM,ECOSYSTEM_CATEGORY,ECOSYSTEM_TYPE,ECOSYSTEM_SUBTYPE,...,ALTITUDE,ALTITUDE2,SOLUBLE_IRON_MICROMOL,BICARBONATE_MILLIMOL,H2S_MILLIMOL,H2S_PRESENT,IRRADIANCE,OXYGEN_PRESENCE,METHANE_CONC_MILLIMOL,path
7,173867,Freshwater microbial communities from Amazon R...,09-DEC-17 11.43.44.472000000 PM,05-JAN-18 12.54.24.000000000 PM,102151,Freshwater microbial communities from Amazon R...,Environmental,Aquatic,Freshwater,River,...,,,,,,,,,,Environmental:Aquatic:Freshwater:River
8,173872,Freshwater microbial communities from Amazon R...,09-DEC-17 11.44.06.993000000 PM,05-JAN-18 12.54.24.000000000 PM,102151,Freshwater microbial communities from Amazon R...,Environmental,Aquatic,Freshwater,River,...,,,,,,,,,,Environmental:Aquatic:Freshwater:River
9,173903,Lake sediment microbial communtites from St. P...,09-DEC-17 11.47.38.822000000 PM,20-APR-18 06.00.14.000000000 PM,101232,Lake sediment microbial communtites from St. P...,Environmental,Aquatic,Freshwater,Lake,...,,,,,,,,,,Environmental:Aquatic:Freshwater:Lake:Sediment
61,156244,Sorted cell/s from the Aspo Hard Rock Laborato...,21-JUL-17 12.50.31.216000000 PM,,,Sorted cells from the Aspo Hard Rock Laborator...,Environmental,Aquatic,Freshwater,Groundwater,...,,,,,,,,,,Environmental:Aquatic:Freshwater:Groundwater
62,156255,Sorted cell/s from the Aspo Hard Rock Laborato...,21-JUL-17 12.50.36.771000000 PM,,,Sorted cells from the Aspo Hard Rock Laborator...,Environmental,Aquatic,Freshwater,Groundwater,...,,,,,,,,,,Environmental:Aquatic:Freshwater:Groundwater
63,156276,Sorted cell/s from the Aspo Hard Rock Laborato...,21-JUL-17 12.50.47.524000000 PM,,,Sorted cells from the Aspo Hard Rock Laborator...,Environmental,Aquatic,Freshwater,Groundwater,...,,,,,,,,,,Environmental:Aquatic:Freshwater:Groundwater
64,156280,Sorted cell/s from the Aspo Hard Rock Laborato...,21-JUL-17 12.50.49.664000000 PM,,,Sorted cells from the Aspo Hard Rock Laborator...,Environmental,Aquatic,Freshwater,Groundwater,...,,,,,,,,,,Environmental:Aquatic:Freshwater:Groundwater
65,156281,Sorted cell/s from the Aspo Hard Rock Laborato...,21-JUL-17 12.50.50.190000000 PM,,,Sorted cells from the Aspo Hard Rock Laborator...,Environmental,Aquatic,Freshwater,Groundwater,...,,,,,,,,,,Environmental:Aquatic:Freshwater:Groundwater
66,156288,Sorted cell/s from the Aspo Hard Rock Laborato...,21-JUL-17 12.50.53.934000000 PM,,,Sorted cells from the Aspo Hard Rock Laborator...,Environmental,Aquatic,Freshwater,Groundwater,...,,,,,,,,,,Environmental:Aquatic:Freshwater:Groundwater
67,156295,Sorted cell/s from the Aspo Hard Rock Laborato...,21-JUL-17 12.50.57.631000000 PM,,,Sorted cells from the Aspo Hard Rock Laborator...,Environmental,Aquatic,Freshwater,Groundwater,...,,,,,,,,,,Environmental:Aquatic:Freshwater:Groundwater


In [110]:
set(depth_df.ECOSYSTEM_CATEGORY)

{'Air',
 'Algae',
 'Annelida',
 'Aquatic',
 'Arthropoda',
 'Bioreactor',
 'Bioremediation',
 'Biotransformation',
 'Built environment',
 'Cnidaria',
 'Food production',
 'Fungi',
 'Human',
 'Insecta',
 'Invertebrates',
 'Lab enrichment',
 'Mammals',
 'Microbial',
 'Modeled',
 'Mollusca',
 'Plants',
 'Porifera',
 'Solid waste',
 'Terrestrial',
 'Tunicates',
 'Unclassified',
 'Wastewater'}